In [1]:
import numpy as onp
import tensorflow as tf
from functools import partial

# Attacking
from cleverhans.utils import clip_eta, one_hot

# Plotting
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils import *

sns.set_style(style='white')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
DATASET = 'cifar10'
class_num   = 10
image_shape = None

train_size = None
test_size  = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

In [4]:
x_train, y_train, x_test, y_test = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                           do_flatten_and_normalize=False))

In [5]:
x_train, x_test = x_train.reshape(-1, *image_shape), x_test.reshape(-1, *image_shape)
x_train, x_test = x_train*255, x_test*255
x_train, x_test = x_train.astype(onp.float32), x_test.astype(onp.float32)

In [6]:
x_train = tf.keras.applications.vgg16.preprocess_input(x_train)
x_test  = tf.keras.applications.vgg16.preprocess_input(x_test)

In [8]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights=None, classes=10, input_shape=(32, 32, 3))

In [9]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [10]:
vgg16.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
vgg16.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

Epoch 1/100
1407/1407 [==============================] - 21s 15ms/step - loss: 2.3622 - accuracy: 0.0989 - val_loss: 2.3518 - val_accuracy: 0.1094
Epoch 2/100
1407/1407 [==============================] - 16s 11ms/step - loss: 2.3622 - accuracy: 0.0990 - val_loss: 2.3518 - val_accuracy: 0.1094
Epoch 3/100
1407/1407 [==============================] - 21s 15ms/step - loss: 2.3622 - accuracy: 0.0990 - val_loss: 2.3518 - val_accuracy: 0.1094
Epoch 4/100
1407/1407 [==============================] - 16s 11ms/step - loss: 2.3622 - accuracy: 0.0990 - val_loss: 2.3518 - val_accuracy: 0.1094
Epoch 5/100
1407/1407 [==============================] - 16s 11ms/step - loss: 2.3622 - accuracy: 0.0990 - val_loss: 2.3518 - val_accuracy: 0.1094
Epoch 6/100
1407/1407 [==============================] - 16s 11ms/step - loss: 2.3622 - accuracy: 0.0990 - val_loss: 2.3518 - val_accuracy: 0.1094
Epoch 7/100
1407/1407 [==============================] - 16s 12ms/step - loss: 2.3622 - accuracy: 0.0990 - val_loss: 2

KeyboardInterrupt: 